In [3]:
# Importando los módulos que necesitaremos
import numpy as np

import pandas as pd
from pandas.plotting import table

import matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib.ticker import PercentFormatter

import seaborn as sns

from datetime import datetime as DateTime

# Configurando los estílos de los gráficos
plt.ioff()
sns.set_context('talk')
sns.set_style("whitegrid")

# Definiendo Constantes
BINS = [0, 10, 20, 30, 40, 50, 60, 70, 80, 300]

## Limpieza de Datos

In [4]:
# Leo la fuente de datos local en CSV 
df = pd.read_csv('data/vacunation_data.csv')
df.head()

,FECHA_CORTE,UUID,GRUPO_RIESGO,EDAD,SEXO,FECHA_VACUNACION,DOSIS,FABRICANTE,DIRESA,DEPARTAMENTO,PROVINCIA,DISTRITO
0,20210506,04cd9264959046894f4235d5c8b62794,PERSONAL DE SALUD,NaN,MASCULINO,20210222,1,SINOPHARM,LIMA CENTRO,LIMA,LIMA,LA VICTORIA
1,20210506,14fd8ec748ba82a72a4c5556ee8e6372,PERSONAL DE SALUD,NaN,MASCULINO,20210310,1,SINOPHARM,LA LIBERTAD,LA LIBERTAD,TRUJILLO,VICTOR LARCO HERRERA
2,20210506,b17ceddbbde5df1c082e09dc076421fb,PERSONAL DE SALUD,NaN,MASCULINO,20210227,1,SINOPHARM,HUANUCO,HUANUCO,LEONCIO PRADO,RUPA-RUPA
3,20210506,e5e67a589a53167d365e3de4373c419d,ADULTO MAYOR,NaN,FEMENINO,20210417,1,PFIZER,LIMA SUR,LIMA,LIMA,CHORRILLOS
4,20210506,103ae79bc8e63d82e78c754ae5c78e75,ADULTO MAYOR,81.0,FEMENINO,20210422,1,ASTRAZENECA,UCAYALI,UCAYALI,CORONEL PORTILLO,CALLERIA


In [5]:
# Obtengo el conteo del número de valores en cada columnas
total_count = len(df)
print(f'TOTAL COUNT: {total_count}')
print(df.count())

TOTAL COUNT: 1939155
FECHA_CORTE         1939155
UUID                1939155
GRUPO_RIESGO        1939155
EDAD                1816807
SEXO                1939155
FECHA_VACUNACION    1939155
DOSIS               1939155
FABRICANTE          1939155
DIRESA              1939155
DEPARTAMENTO        1939155
PROVINCIA           1939155
DISTRITO            1939155
dtype: int64


In [6]:
# Verificamos que las columnas DEPARTAMENTO, GRUPO_RIESGO y SEXO tengan los valores indicados
print(df['DEPARTAMENTO'].unique())
print(df['GRUPO_RIESGO'].unique())
print(df['SEXO'].unique())

['LIMA' 'LA LIBERTAD' 'HUANUCO' 'UCAYALI' 'SAN MARTIN' 'LORETO' 'PASCO'
 'CALLAO' 'ICA' 'PIURA' 'JUNIN' 'ANCASH' 'AYACUCHO' 'HUANCAVELICA'
 'LAMBAYEQUE' 'APURIMAC' 'CAJAMARCA' 'TUMBES' 'AMAZONAS' 'AREQUIPA'
 'MADRE DE DIOS' 'CUSCO' 'TACNA' 'MOQUEGUA' 'PUNO']
['PERSONAL DE SALUD' 'ADULTO MAYOR' 'PERSONAL MILITAR Ó FF AA'
 'BRIGADISTAS' 'TRABAJADOR Ó PERSONAL DE LIMPIEZA'
 'POLICIA NACIONAL DEL PERU' 'PERSONAL DE SEGURIDAD' 'BOMBERO'
 'ESTUDIANTES DE CIENCIAS DE LA SALUD']
['MASCULINO' 'FEMENINO']


In [7]:
# Nos quedamos con los datos que correspondan solo a la primera dosis
df = df[df['DOSIS'] == 1]

In [8]:
# Filtramos las filas que no tienen valor en las columnas EDAD o SEXO
df = df[df['EDAD'].notnull() & df['SEXO'].notnull()]

## Transformación de Datos

In [9]:
# Abreviando los grupos de riesgo
df['GRUPO_RIESGO'].replace('PERSONAL DE SALUD','P. SALUD', inplace=True)
df['GRUPO_RIESGO'].replace('TRABAJADOR Ó PERSONAL DE LIMPIEZA','T. LIMP.', inplace=True)
df['GRUPO_RIESGO'].replace('PERSONAL DE SEGURIDAD','P. SEG.', inplace=True)
df['GRUPO_RIESGO'].replace('PERSONAL MILITAR Ó FF AA','FF.AA.', inplace=True)
df['GRUPO_RIESGO'].replace('POLICIA NACIONAL DEL PERU','P.N.P.', inplace=True)
df['GRUPO_RIESGO'].replace('ESTUDIANTES DE CIENCIAS DE LA SALUD','E. SALUD', inplace=True)
df['GRUPO_RIESGO'].replace('BRIGADISTAS','BRIG.', inplace=True)

# Genero Serie de grupos de riesgo
grupos_riesgo = df['GRUPO_RIESGO'].unique()
grupos_riesgo_series = pd.Series(np.zeros_like(grupos_riesgo),grupos_riesgo)

# Agrupo el conjunto de datos por DEPARTAMENTO, SEXO y EDAD para el DataFrame
gb_age = df[["DEPARTAMENTO","SEXO",'EDAD']].groupby(['DEPARTAMENTO', 'SEXO', pd.cut(df['EDAD'], BINS, False)])

# Agrupo el conjunto de datos por DEPARTAMENTO, SEXO y GRUPO_RIESGO para el DataFrame
gb_gr = df[["DEPARTAMENTO","SEXO",'GRUPO_RIESGO']].groupby(['DEPARTAMENTO', 'SEXO', 'GRUPO_RIESGO'])

In [10]:
def get_data_to_plot(department=None):
    if(department is None):
        return df[['SEXO','EDAD','GRUPO_RIESGO','FECHA_VACUNACION']]
    else:
        return df[df['DEPARTAMENTO'] == department][['SEXO','EDAD','GRUPO_RIESGO','FECHA_VACUNACION']]

## Visualización de Datos

In [11]:
# Visualizo los datos agrupados por EDAD via DataFrame
gb_age.size().unstack()

EDAD                     [0, 10)  [10, 20)  [20, 30)  [30, 40)  [40, 50)  \
DEPARTAMENTO  SEXO                                                         
AMAZONAS      FEMENINO         0        14       910      1348       762   
              MASCULINO        0        14      1015      1083       639   
ANCASH        FEMENINO         0        13      2499      3431      2137   
              MASCULINO        0        80      2833      2621      1404   
APURIMAC      FEMENINO         0         5      1185      1885      1292   
              MASCULINO        0         4      1036      1296       958   
AREQUIPA      FEMENINO         0        72      2787      4664      3236   
              MASCULINO        0       748      3914      3414      2638   
AYACUCHO      FEMENINO         0         9      1287      2167      1477   
              MASCULINO        0         8       705      1560      1148   
CAJAMARCA     FEMENINO         0         8      2253      3399      2022   
              MASCULINO        0        16      1048      2438      1420   
CALLAO        FEMENINO         0       393      4114      5882      4154   
              MASCULINO        0      1500     12086      8635      7775   
CUSCO         FEMENINO         0        15      1405      3314      2179   
              MASCULINO        0        14       620      1766      1799   
HUANCAVELICA  FEMENINO         0        10      1245      1566       862   
              MASCULINO        0        28      1014      1125       717   
HUANUCO       FEMENINO         0         7      1434      2409      1423   
              MASCULINO        0        17       811      1982      1033   
ICA           FEMENINO         0        29      2485      3092      2237   
              MASCULINO        0        47      2008      2083      1534   
JUNIN         FEMENINO         0         8      2518      3990      2331   
              MASCULINO        0        52       936      2437      1835   
LA LIBERTAD   FEMENINO         0         7      2776      4177      2657   
              MASCULINO        0        38      2911      2780      1777   
LAMBAYEQUE    FEMENINO         0        15      1669      2920      2220   
              MASCULINO        0        29       804      1998      1728   
LIMA          FEMENINO         0       801     27973     47502     33940   
              MASCULINO        0      6338     40194     35199     28209   
LORETO        FEMENINO         0        49      1141      2223      1357   
              MASCULINO        0       462      2486      2250      1529   
MADRE DE DIOS FEMENINO         0        11       358       594       377   
              MASCULINO        0         5       468       559       293   
MOQUEGUA      FEMENINO         0         5       643       943       729   
              MASCULINO        0        13       519       577       419   
PASCO         FEMENINO         0         4       601       755       554   
              MASCULINO        0        14       668       537       395   
PIURA         FEMENINO         0        70      3200      4402      3275   
              MASCULINO        0        96      1972      3230      2226   
PUNO          FEMENINO         0        19      1900      2807      1750   
              MASCULINO        0        10      1019      2023      1413   
SAN MARTIN    FEMENINO         0        36      1592      2259      1065   
              MASCULINO        0        36      1917      1891       935   
TACNA         FEMENINO         0        20       723      1180       933   
              MASCULINO        0       233      1092      1102       727   
TUMBES        FEMENINO         0        15       638       718       549   
              MASCULINO        0       401      1860       773       497   
UCAYALI       FEMENINO         0        26      1071      1365       885   
              MASCULINO        0        84      1241      1228       761   

EDAD                     [50, 60)  [60, 70)  [7

In [12]:
# Visualizo los datos agrupados por GRUPO_RIESGO via DataFrame
gb_gr.size().unstack()

GRUPO_RIESGO             ADULTO MAYOR  BOMBERO  BRIG.  E. SALUD   FF.AA.  \
DEPARTAMENTO  SEXO                                                         
AMAZONAS      FEMENINO         4677.0      1.0    9.0       5.0     10.0   
              MASCULINO        4443.0     18.0   25.0       6.0     17.0   
ANCASH        FEMENINO         4288.0     36.0    NaN      46.0      7.0   
              MASCULINO        3903.0    112.0    NaN      26.0    175.0   
APURIMAC      FEMENINO        10712.0     16.0   44.0       NaN      1.0   
              MASCULINO        8741.0     53.0   36.0       NaN      5.0   
AREQUIPA      FEMENINO         9374.0      NaN    4.0       2.0    493.0   
              MASCULINO        7362.0      NaN    NaN       3.0   3624.0   
AYACUCHO      FEMENINO        14354.0      7.0   44.0       6.0      NaN   
              MASCULINO       11054.0     16.0   30.0       3.0      NaN   
CAJAMARCA     FEMENINO         4827.0      NaN    NaN      15.0      NaN   
              MASCULINO        3875.0      NaN    1.0      12.0      2.0   
CALLAO        FEMENINO        20972.0    221.0   27.0      31.0   3629.0   
              MASCULINO       17267.0    699.0   34.0      89.0  21944.0   
CUSCO         FEMENINO         3271.0      NaN    NaN       8.0      NaN   
              MASCULINO        2611.0      NaN    NaN       4.0      5.0   
HUANCAVELICA  FEMENINO         9850.0     17.0   71.0       NaN      NaN   
              MASCULINO        8285.0     52.0   65.0       NaN     65.0   
HUANUCO       FEMENINO         8610.0      1.0   78.0       5.0      NaN   
              MASCULINO        7927.0      NaN   78.0       3.0      NaN   
ICA           FEMENINO         5926.0     96.0    NaN      55.0     99.0   
              MASCULINO        4731.0    257.0    NaN      24.0    299.0   
JUNIN         FEMENINO         5908.0     26.0    3.0     170.0     53.0   
              MASCULINO        4851.0     74.0    NaN      50.0    229.0   
LA LIBERTAD   FEMENINO        14276.0      1.0   15.0     168.0      NaN   
              MASCULINO       11736.0      NaN   10.0      65.0      NaN   
LAMBAYEQUE    FEMENINO         5049.0      NaN    NaN       NaN     97.0   
              MASCULINO        3874.0      NaN    NaN       5.0    301.0   
LIMA          FEMENINO       151094.0   1271.0  297.0     109.0   3629.0   
              MASCULINO      117889.0   3035.0  252.0     138.0  36071.0   
LORETO        FEMENINO         6532.0     26.0   33.0      22.0    139.0   
              MASCULINO        6022.0    129.0   23.0      12.0   2388.0   
MADRE DE DIOS FEMENINO          841.0     13.0    NaN       1.0      6.0   
              MASCULINO        1171.0     52.0    NaN       NaN     14.0   
MOQUEGUA      FEMENINO          546.0      NaN    NaN       NaN      NaN   
              MASCULINO         621.0      NaN    NaN       NaN      NaN   
PASCO         FEMENINO         3654.0      NaN    9.0       2.0      2.0   
              MASCULINO        3240.0      NaN   11.0       NaN      2.0   
PIURA         FEMENINO         7587.0     45.0    1.0      17.0    213.0   
              MASCULINO        6148.0    271.0    2.0      19.0    786.0   
PUNO          FEMENINO         2211.0      NaN    NaN     401.0      3.0   
              MASCULINO        2058.0      NaN    NaN      95.0     14.0   
SAN MARTIN    FEMENINO         4069.0      NaN    NaN     129.0      NaN   
              MASCULINO        3306.0      NaN    1.0      56.0      2.0   
TACNA         FEMENINO         1884.0    107.0    NaN       9.0     71.0   
              MASCULINO        1749.0    240.0    NaN       7.0   1090.0   
TUMBES        FEMENINO         3197.0      NaN    9.0      41.0     98.0   
              MASCULINO        3498.0      NaN   10.0       8.0   1808.0   
UCAYALI       FEMENINO         3720.0     12.0    8.0       NaN     15.0   
              MASCULINO        4361.0    131.0   15.0       NaN    328.0   

GRUPO_RIESGO             P. SALUD  P. SEG.   P.

In [13]:
yticks = [5,15,25,35,45,55,65,75,85]

def generate_plot(area, data, date_range, show_plot=False, save_plot=True):

    # Creamos la figura, los ejes y agregamos la atribución
    fig, axs = plt.subplots(2,2, figsize=(20,20))
    plt.figtext(0.1,0.95, area + ': DISTRIBUCIÓN DE VACUNADOS CONTRA COVID-19 (N=' + str(len(data)) + ')', ha='left', fontsize=24)
    plt.figtext(0.1,0.925, 'Rango de fechas de vacunación: del {} al {}'.format(f'{date_range[0]:%Y-%m-%d}',f'{date_range[1]:%Y-%m-%d}'), ha='left', fontsize=20, color='#999')
    plt.figtext(0.9,0.02, """Fuente: https://www.datosabiertos.gob.pe/dataset/vacunaci%C3%B3n-contra-covid-19-ministerio-de-salud-minsa
    Autor: https://malexandersalazar.github.io/, Moisés Alexander Salazar Vila, """ + f'{DateTime.now():%Y-%m-%d}', ha='right')

    # 1. Graficamos distribución por GRUPO ETAREO
    xlim_max = 0

    ## 1.1. Graficamos el primer histograma
    N, bins, patches = axs[0][0].hist(data["EDAD"], BINS, orientation = 'horizontal')

    ### Colorearemos las barras en base el conteo relativo de cada agrupación
    xlim_max = N.max() * 1.1
    fracs = N / N.max()
    norm = colors.Normalize(fracs.min(), fracs.max())

    ### Recorremos las barras y actualizamos cada color respectivamente
    for thisfrac, thispatch in zip(fracs, patches):
        color = plt.cm.viridis(norm(thisfrac))
        thispatch.set_facecolor(color)

    ### Personalizamos las ejes de nuestro gráfico
    axs[0][0].set_yticklabels([])
    axs[0][0].yaxis.set_tick_params(width=0)
    axs[0][0].set_yticks(yticks, minor=True)
    axs[0][0].set_yticklabels(['0-9','10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70-79', '80+'], minor=True)
    axs[0][0].set_ylim(0,90)
    axs[0][0].set_xlim(0,xlim_max)
    axs[0][0].set_ylabel('GRUPO ETÁREO (AÑOS)')

    ### Calculamos y anotamos los porcentajes correspondientes a cada barra
    for i in range(len(BINS)-1):
        axs[0][0].text(N[i], yticks[i]-1, str(round(N[i]*100/len(data),2)) + '%')

    ## 1.2. Graficamos el segundo histograma
    males = data[data['SEXO'] == 'MASCULINO']["EDAD"]
    females = data[data['SEXO'] == 'FEMENINO']["EDAD"]

    N, bins, patches = axs[0][1].hist([males, females], bins=9, range=[0,90], orientation = 'horizontal')
    
    ### Personalizamos las ejes de nuestro gráfico
    axs[0][1].set_yticklabels([])
    axs[0][1].yaxis.set_tick_params(width=0)
    axs[0][1].set_yticks(yticks, minor=True)
    axs[0][1].set_ylim(0,90)
    axs[0][1].set_xlim(0,xlim_max)

    ### Calculamos y anotamos los porcentajes correspondientes a cada barra
    for i in range(len(BINS)-1):
        axs[0][1].text(N[0][i], yticks[i]-3, str(round(N[0][i]*100/len(data),2)) + '%')
        axs[0][1].text(N[1][i], yticks[i]+1, str(round(N[1][i]*100/len(data),2)) + '%')

    axs[0][1].legend(['Hombres (n=' + str(len(males)) + ')', 'Mujeres (n=' + str(len(females)) + ')'])

    # 2. Graficamos distribución por GRUPO RIESGO
    xlim_max = 0

    ## 2.1. Graficamos el primer gráfico de barras 
    group_counts = data['GRUPO_RIESGO'].value_counts().add(grupos_riesgo_series,fill_value=0)
    group_counts.sort_index(inplace=True,ascending=False)
    pos = np.array(range(len(group_counts)))
    patches = axs[1][0].barh(pos, group_counts.values, align='center')
    
    ### Colorearemos las barras en base el conteo relativo de cada agrupación
    xlim_max = group_counts.values.max() * 1.1
    fracs = group_counts.values / group_counts.values.max()
    norm = colors.Normalize(fracs.min(), fracs.max())

    ### Recorremos las barras y actualizamos cada color respectivamente
    for thisfrac, thispatch in zip(fracs, patches):
        color = plt.cm.viridis(norm(thisfrac))
        thispatch.set_facecolor(color)

    ### Personalizamos las ejes de nuestro gráfico
    axs[1][0].set_yticks(pos)
    axs[1][0].set_yticklabels(list(group_counts.index.values))
    axs[1][0].set_xlim(0,xlim_max)

    ### Calculamos y anotamos los porcentajes correspondientes a cada barra
    for i in range(len(pos)):
        axs[1][0].text(group_counts.values[i], pos[i]-0.1, str(round(group_counts[i]*100/len(data),2)) + '%')

    ## 2.2. Graficamos el segundo gráfico de barras 
    males = data[data['SEXO'] == 'MASCULINO']
    males_group_counts = males["GRUPO_RIESGO"].value_counts().add(grupos_riesgo_series,fill_value=0)
    males_group_counts.sort_index(inplace=True,ascending=False)

    females = data[data['SEXO'] == 'FEMENINO']
    females_group_counts = females["GRUPO_RIESGO"].value_counts().add(grupos_riesgo_series,fill_value=0)
    females_group_counts.sort_index(inplace=True,ascending=False)

    pos = np.array(range(len(males_group_counts)))

    axs[1][1].barh(pos, males_group_counts.values, 0.35, align='center')
    axs[1][1].barh(pos+0.35, females_group_counts.values, 0.35, align='center')
    
    ### Personalizamos las ejes de nuestro gráfico
    axs[1][1].set_yticks(pos)
    axs[1][1].set_yticklabels([])
    axs[1][1].set_xlim(0,xlim_max)
    
    ### Calculamos y anotamos los porcentajes correspondientes a cada barra
    for i in range(len(pos)):
        axs[1][1].text(males_group_counts.values[i], pos[i]-0.1, str(round(males_group_counts[i]*100/len(data),2)) + '%')
        axs[1][1].text(females_group_counts.values[i], pos[i]+0.3, str(round(females_group_counts[i]*100/len(data),2)) + '%')

    ## 3. Mostramos, guardamos y generamos el Markdown para las imágenes

    sns.despine(left=False, bottom=False)

    if save_plot:
        filename = 'dist/{}_{}.png'.format(f'{DateTime.now():%Y%m%d}', area.replace(' ', '_'))
        plt.savefig("../" + filename, bbox_inches='tight')
        with open("../dist/images.txt", "a", encoding='utf-8') as f:
            f.write('![alt text]({} "{}")\n\n'.format(filename, area))

    if show_plot:
        plt.show()

    plt.close(fig)

In [14]:
# Eliminando archivo generado para el Markdown de las imágenes
if os.path.exists("../dist/images.txt"):
    os.remove("../dist/images.txt")

In [15]:
# Perú
data = get_data_to_plot()
dates = pd.to_datetime(data['FECHA_VACUNACION'], infer_datetime_format=False, format='%Y%m%d')
generate_plot('PERÚ', data, (dates.min(), dates.max()), False, True)

In [16]:
# Departamentos
departments_ordered = np.sort(df['DEPARTAMENTO'].unique())
for d in departments_ordered:
    data = get_data_to_plot(d)
    dates = pd.to_datetime(data['FECHA_VACUNACION'], infer_datetime_format=False, format='%Y%m%d')
    generate_plot(d, data, (dates.min(), dates.max()), False, True)